In [ ]:
!pip install crawlee

In [ ]:
!pip install playwright

In [ ]:
!sudo apt update && sudo apt install -y \
    libgtk-4-bin libgtk-4-dev \
    libgraphene-1.0-0 \
    woff2 \
    libgstreamer-gl1.0-0 \
    libgstreamer-plugins-bad1.0-0 \
    libavif13 \
    libharfbuzz-icu0 \
    libenchant-2-2 \
    libsecret-1-0 \
    libhyphen0 \
    libmanette-0.2-0

Password:
sudo: a password is required


In [ ]:
!playwright install

In [15]:
!pip install googlesearch

ERROR: Could not find a version that satisfies the requirement googlesearch (from versions: none)
ERROR: No matching distribution found for googlesearch


In [16]:
import googlesearch
print(googlesearch.__file__)

!pip uninstall googlesearch

!pip install googlesearch-python

/opt/anaconda3/envs/yahoofinance/lib/python3.12/site-packages/googlesearch/__init__.py


In [ ]:
pip install googlesearch-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
import os
import time
import html
import json
from urllib.parse import urlparse
from pathlib import Path
from crawlee.crawlers import PlaywrightCrawler, PlaywrightCrawlingContext
from googlesearch import search

# For scrapping
# pip install googlesearch-python
# python -m pip install 'crawlee[all]'
# playwright install
# pip install beautifulsoup4

# For labelling dataset
# pip install -U label-studio
# label-studio start

PHRASE = "AES Solar  v. Kingdom of Spain"
TOP_N = 10
output_directory = f"scraped_content/{PHRASE.replace(' ', '_')[:10]}"

def get_top_urls(query, num_results):
    urls = []
    for url in search(query, num_results=num_results):
        urls.append(url)
        print(f"Found URL: {url}")
        time.sleep(0.1)
    return urls

def sanitize_filename(url):
    parsed = urlparse(url)
    netloc = parsed.netloc
    if (netloc.startswith('www.')):
        netloc = netloc[4:]
    filename = netloc + parsed.path.replace('/', '_')
    filename = ''.join(c if c.isalnum() or c in ['_', '-', '.'] else '_' for c in filename)
    return filename[:10]

def create_label_studio_xml(html_content, url, title):
    json_content = json.dumps([{"html_content": html_content}])
    xml_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<View>
  <HyperText name="text" value="$html_content" valueType="text" />
  <Labels name="labels" toName="text">
    <Label value="Important" />
    <Label value="Review" />
  </Labels>
  <Meta>
    <Info name="url" value="{html.escape(url)}"/>
    <Info name="title" value="{html.escape(title)}"/>
  </Meta>
</View>
"""
    return xml_content, json_content

async def main() -> None:
    output_path = Path(output_directory)
    output_path.mkdir(parents=True, exist_ok=True)

    html_output_path = output_path / "html"
    html_output_path.mkdir(exist_ok=True)

    json_output_path = output_path / "json"
    json_output_path.mkdir(exist_ok=True)

    print(f"Searching Google for: {PHRASE}")
    urls = get_top_urls(PHRASE, TOP_N)

    crawler = PlaywrightCrawler(
        headless=True,
    )

    @crawler.router.default_handler
    async def request_handler(context: PlaywrightCrawlingContext) -> None:
        url = context.request.url
        context.log.info(f'Processing {url} ...')

        await context.page.wait_for_load_state("networkidle", timeout=30000)
        html_content = await context.page.content()
        title = await context.page.title()
        _, json_content = create_label_studio_xml(html_content, url, title)

        filename = sanitize_filename(url)

        json_filename = filename + '.json'
        json_file_path = json_output_path / json_filename
        with open(json_file_path, 'w', encoding='utf-8') as f:
            json.dump([{"html_content": html_content}], f, ensure_ascii=False, indent=4)
        context.log.info(f'Saved JSON content to {json_file_path}')

        html_filename = filename + '.html'
        html_file_path = html_output_path / html_filename
        with open(html_file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        context.log.info(f'Saved HTML content to {html_file_path}')

        data = {
            'url': url,
            'title': title,
            'saved_json': str(json_file_path),
            'saved_html': str(html_file_path),
        }
        await context.push_data(data)

    await crawler.run(urls)
    print(f"Scraping completed. Files saved to:")
    print(f"- JSON: {output_directory}/json/")
    print(f"- HTML: {output_directory}/html/")
    print(f"You can now import the JSON files into Label Studio for annotation.")

if __name__ == '__main__':
    asyncio.run(main())

Searching Google for: AES Solar  v. Kingdom of Spain
Found URL: https://jusmundi.com/en/document/decision/en-aes-solar-and-others-pv-investors-v-the-kingdom-of-spain-order-of-the-united-states-court-of-appeals-for-the-district-of-columbia-friday-5th-january-2024
Found URL: https://www.italaw.com/cases/9887
Found URL: https://pca-cpa.org/en/cases/238/
Found URL: https://jusmundi.com/en/document/decision/nl-aes-solar-and-others-pv-investors-v-the-kingdom-of-spain-uitspraak-van-het-rechtbank-amsterdam-wednesday-15th-march-2023
Found URL: https://www.transnational-dispute-management.com/legal-and-regulatory-detail.asp?key=33041
Found URL: https://www.italaw.com/cases/documents/13509
Found URL: https://law.justia.com/cases/federal/district-courts/district-of-columbia/dcdce/1:2021cv03249/238369/44/
Found URL: https://www.transnational-dispute-management.com/legal-and-regulatory-detail.asp?key=33042
Found URL: https://arbitration.org/award/1048
Found URL: https://investmentpolicy.unctad.org/i

[crawlee.crawlers._playwright._playwright_crawler] INFO  Current request statistics:
┌───────────────────────────────┬──────────┐
│ requests_finished             │ 0        │
│ requests_failed               │ 0        │
│ retry_histogram               │ [0]      │
│ request_avg_failed_duration   │ None     │
│ request_avg_finished_duration │ None     │
│ requests_finished_per_minute  │ 0        │
│ requests_failed_per_minute    │ 0        │
│ request_total_duration        │ 0.0      │
│ requests_total                │ 0        │
│ crawler_runtime               │ 0.016209 │
└───────────────────────────────┴──────────┘
[crawlee._autoscaling.autoscaled_pool] INFO  current_concurrency = 0; desired_concurrency = 2; cpu = 0.0; mem = 0.0; event_loop = 0.0; client_info = 0.0
[crawlee._autoscaling.autoscaled_pool] INFO  Waiting for remaining tasks to finish


[crawlee.crawlers._playwright._playwright_crawler] INFO  Final request statistics:
┌───────────────────────────────┬─────────┐
│ requests_finished             │ 0       │
│ requests_failed               │ 0       │
│ retry_histogram               │ [0]     │
│ request_avg_failed_duration   │ None    │
│ request_avg_finished_duration │ None    │
│ requests_finished_per_minute  │ 0       │
│ requests_failed_per_minute    │ 0       │
│ request_total_duration        │ 0.0     │
│ requests_total                │ 0       │
│ crawler_runtime               │ 0.29683 │
└───────────────────────────────┴─────────┘


Scraping completed. Files saved to:
- JSON: scraped_content/AES_Solar_/json/
- HTML: scraped_content/AES_Solar_/html/
You can now import the JSON files into Label Studio for annotation.
